## Calculate metadata and create point file from geopackage files

In [1]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from shapely.wkt import loads
from osgeo import osr
from pyproj import CRS
import math
import warnings

# Function to reproject raster to match CRS of polygons
def reproject_raster_to_match_polygons(src, polygons_crs):
    if src.crs != polygons_crs:
        # Reproject raster to match polygons CRS
        transform, width, height = calculate_default_transform(src.crs, polygons_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': polygons_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        reprojected_data = np.empty((src.count, height, width), dtype=src.dtypes[0])
        reproject(
            source=src.read(),
            destination=reprojected_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=polygons_crs,
            resampling=Resampling.nearest
        )

        return reprojected_data, kwargs
    else:
        return '_', src.meta

# Radius of Mars in kilometers at Argyre (assuming ellipsoid with 3396.19, 3376.2)
R_MARS = 3384.5

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Mars (in kilometers)
    given their latitudes and longitudes.
    
    :param lat1: Latitude of the first point in degrees
    :param lon1: Longitude of the first point in degrees
    :param lat2: Latitude of the second point in degrees
    :param lon2: Longitude of the second point in degrees
    :return: Distance between the two points in kilometers
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    d = R_MARS * c
    return d

def custom_distance(p1, p2):
    lon1, lat1 = p1.x, p1.y
    lon2, lat2 = p2.x, p2.y
    return round(haversine(lon1, lat1, lon2, lat2), 2)

# Function to calculate representative point of each polygon
def calculate_representative_point(polygons, key_):
    polygons[f'representative_point_{key_}'] = polygons['geometry'].representative_point()
    return polygons

# Function to calculate mean of raster over each polygon
def calculate_metadata(polygons, src1,  src2, geounits, single_point, source_file, crs):
    # Suppress specific runtime warnings
    warnings.filterwarnings('ignore', category=RuntimeWarning)

    # List to store results
    results = []

    # Loop over each polygon
    for idx, row in tqdm(polygons.iterrows(), total=len(polygons), desc="Calculating mean raster"):
        # Mask raster by each polygon
        try:
            out_image, out_transform = mask(src1, [row.geometry], crop=True)
            ti_image, ti_transform = mask(src2, [row.geometry], crop=True)
        except ValueError:
            continue
        
        out_image = np.where(out_image < -8100, np.nan, out_image)
        ti_image = np.where(ti_image < 0, np.nan, ti_image)

        # Calculate mean
        with np.errstate(invalid='ignore', divide='ignore'):
            mean_value = np.nanmean(out_image)
            mean_ti = np.nanmean(ti_image)

        # Calculate distance from polygon's representative point to single point
        p = row['representative_point_distance']
        
        distance = custom_distance(single_point, p)

        cat, con = row['Category'].split('(')
        con = con.replace(')', '') 
        cat = cat.strip()
        con = con.strip()

        geounits_gdf = gpd.read_file(geounits)
        p = row['representative_point_units']
        intersecting_features = geounits_gdf[geounits_gdf.intersects(p)]
        try:
            unit_name = intersecting_features['UnitName'].values[0]
        except:
            unit_name = 'None'
        # if idx % 10 == 0:
        #     print(f"Unit name: {unit_name}")
        # Append results
        results.append({
            'Source File': source_file,
            'Polygon_ID': idx,
            'Polygon Number': row['Polygon Number'],
            'Color': row['Color'],
            'Mineral ID 1': row['Mineral ID 1'].lower(),
            'Mineral ID 2': row['Mineral ID 2'].lower(),
            'Mineral ID 3': row['Mineral ID 3'].lower(),
            'Mineral ID 4': row['Mineral ID 4'].lower(),
            'Category': cat.lower(),
            'Confidence': con.lower(),
            'Representative Point': row['representative_point_distance'],
            'TI': mean_ti,
            'Elevation_km': mean_value/1000,
            'Distance_to_center_km': distance,
            'GeoUnits': unit_name
        })

    # Create a DataFrame from results
    df = pd.DataFrame(results)

    return df

# Function to save results to Excel
def save_to_excel(df, excel_file):
    df.to_excel(excel_file, index=False, engine='openpyxl')

# Function to process all gpkg files in a folder
def process_gpkg_folder(gpkg_folder, raster_file, raster_file_2, geounits, point_shapefile, output_excel, out_gpkg):
    all_results = []
    # Define Mars CRS
    mars_crs_simple = {
        'proj': 'longlat',
        'ellps': 'sphere',
        'a': 3396190,  # semi-major axis
        'b': 3396190,  # semi-minor axis
        'units': 'm',
        'no_defs': True
    }
    # Path to your .prj file
    prj_file_path = '/Users/phillipsm/Documents/Research/MassifMapping/ArgyreMap/Argyre_3M_preliminary/CoordinateSytems/Argyre_LCC_Mars2000sphere.prj'

    # Function to read the .prj file and return the CRS as a WKT string
    def read_prj_file(prj_file):
        with open(prj_file, 'r') as file:
            prj_text = file.read()
        return prj_text
    
    # Read the .prj file
    prj_text = read_prj_file(prj_file_path)

    # Create an osr.SpatialReference object
    spatial_ref = osr.SpatialReference()
    spatial_ref.ImportFromWkt(prj_text)

    # Convert to pyproj CRS
    mars_crs = CRS.from_wkt(spatial_ref.ExportToWkt())

    # Read the single point from shapefile into a GeoDataFrame
    single_point_gdf = gpd.read_file(point_shapefile)

    # Reproject the GeoDataFrame to the desired CRS
    single_point_gdf = single_point_gdf.to_crs(mars_crs_simple)
    
    # Extract the reprojected geometry
    single_point = single_point_gdf.iloc[0]['geometry']

    # Print single point CRS for debugging
    print(f"Single point CRS: {single_point_gdf.crs}")

    # Open raster file
    with rasterio.open(raster_file) as src1, rasterio.open(raster_file_2) as src2:
        # raster_data, raster_meta = reproject_raster_to_match_polygons(src, polygons_crs=src.crs)

        # List all .gpkg files in the folder
        gpkg_files = [os.path.join(gpkg_folder, file) for file in os.listdir(gpkg_folder) if file.endswith('.gpkg')]

        # Process each .gpkg file
        for gpkg_file in gpkg_files:
            print(f"Processing {gpkg_file}...")
            # Read the polygon geometries from the .gpkg file
            polygons = gpd.read_file(gpkg_file)

            # Reproject polygons to Mars CRS
            polygons = polygons.to_crs(mars_crs)
            print(f"Reprojected CRS: {polygons.crs}")

            # Calculate representative point for each polygon
            p_crs = polygons.crs
            polygons = polygons.to_crs(mars_crs_simple)
            polygons = calculate_representative_point(polygons, key_ = 'distance')
            polygons = polygons.to_crs(p_crs)
            polygons = calculate_representative_point(polygons, key_ = 'units')

            source_file = gpkg_file.split('.')[0].split('/')[-1]
            polygons = polygons.to_crs(src1.crs)  # Ensure polygons are in the raster CRS

            results_df = calculate_metadata(polygons, src1, src2, geounits, single_point, source_file, mars_crs_simple)
            all_results.append(results_df)

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(all_results, ignore_index=True)

    # Save results to Excel
    save_to_excel(final_df, output_excel)

    # Save the updated GeoDataFrame back to the original .gpkg file
    # final_df['geometry'] = final_df['Representative Point']
    gdf = gpd.GeoDataFrame(final_df, geometry = 'Representative Point')
    gdf.set_crs(mars_crs)
    gdf.to_file(out_gpkg, driver='GPKG')

# usage
DATE_STRING = '03062025'
if __name__ == "__main__":
    # Folder containing .gpkg files
    gpkg_folder = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/categorized_olivine_2/'
    # gpkg_folder = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/targeted/categorized/'

    # Path to raster DEM file
    dem_file = '/Volumes/Rohan/Mars_GIS_Data/MOLA_HRSC/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif'
    
    # Path to raster ti file
    ti_file = '/Volumes/Rohan/Mars_GIS_Data/THEMIS/thermal_inertia/THEMIS_TI_Mosaic_Quant_60S300E_100mpp_ESRI104971.tif'

    # Path to single point shapefile
    point_shapefile = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/arygyre_center_point_argyreProj.shp'

    # Path to single point shapefile
    geounits = 'file:///Users/phillipsm/Documents/Research/MassifMapping/ArgyreMap/Argyre_3M_preliminary/symbology_layers/argyre_geo_units_dohm2015.geojson'

    # Output Excel file
    output_excel = f'/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/mineral_unit_metadata_categorized_{DATE_STRING}.xlsx'

    # output geopackage file
    out_gpkg = f'/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/all_units_points_argyrePRJ_{DATE_STRING}.gpkg'
    
    # Process all .gpkg files in the folder
    process_gpkg_folder(gpkg_folder, dem_file, ti_file, geounits, point_shapefile, output_excel, out_gpkg)



/Users/phillipsm/anaconda3/envs/hypy_scat/lib/python3.10/site-packages/osgeo/osr.py:385: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Single point CRS: +proj=longlat +ellps=sphere +a=3396190 +b=3396190 +units=m +no_defs +type=crs
Processing /Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/categorized_olivine_2/T0507.gpkg...
Reprojected CRS: PROJCS["Argyre-Lambert Conformal Conic [clon -45E]",GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0],AUTHORITY["ESRI","106971"]],PRIMEM["Reference_Meridian",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-45],PARAMETER["standard_parallel_1",-35.83333],PARAMETER["standard_parallel_2",-59.16666],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Calculating mean raster:   0%|          | 0/72 [00:00<?, ?it/s]

Processing /Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/categorized_olivine_2/T0360.gpkg...
Reprojected CRS: PROJCS["Argyre-Lambert Conformal Conic [clon -45E]",GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0],AUTHORITY["ESRI","106971"]],PRIMEM["Reference_Meridian",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-45],PARAMETER["standard_parallel_1",-35.83333],PARAMETER["standard_parallel_2",-59.16666],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Calculating mean raster:   0%|          | 0/29 [00:00<?, ?it/s]

...

Calculating mean raster:   0%|          | 0/177 [00:00<?, ?it/s]

Processing /Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/categorized_olivine_2/T0359.gpkg...
Reprojected CRS: PROJCS["Argyre-Lambert Conformal Conic [clon -45E]",GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0],AUTHORITY["ESRI","106971"]],PRIMEM["Reference_Meridian",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-45],PARAMETER["standard_parallel_1",-35.83333],PARAMETER["standard_parallel_2",-59.16666],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Calculating mean raster:   0%|          | 0/38 [00:00<?, ?it/s]

Processing /Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/categorized_olivine_2/T0504.gpkg...
Reprojected CRS: PROJCS["Argyre-Lambert Conformal Conic [clon -45E]",GEOGCS["GCS_Mars_2000_Sphere",DATUM["Mars_2000_(Sphere)",SPHEROID["Mars_2000_Sphere_IAU_IAG",3396190,0],AUTHORITY["ESRI","106971"]],PRIMEM["Reference_Meridian",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-45],PARAMETER["standard_parallel_1",-35.83333],PARAMETER["standard_parallel_2",-59.16666],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


Calculating mean raster:   0%|          | 0/130 [00:00<?, ?it/s]

# translate TI data to DEM crs

In [4]:
raster_file = '/Volumes/Rohan/Mars_GIS_Data/MOLA_HRSC/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif'
raster_file_2 = '/Volumes/Rohan/Mars_GIS_Data/THEMIS/thermal_inertia/THEMIS_TI_Mosaic_Quant_60S300E_100mpp.tif'
with rasterio.open(raster_file) as src1, rasterio.open(raster_file_2) as src2:
    # Reproject src2 to the CRS of src1
    if src2.crs != src1.crs:
        transform, width, height = calculate_default_transform(src2.crs, src1.crs, src2.width, src2.height, *src2.bounds)
        kwargs = src2.meta.copy()
        kwargs.update({
        'crs': src1.crs,
        'transform': transform,
        'width': width,
        'height': height
        })

        reprojected_data = np.empty((src2.count, height, width), dtype=src2.dtypes[0])
        reproject(
        source=src2.read(),
        destination=reprojected_data,
        src_transform=src2.transform,
        src_crs=src2.crs,
        dst_transform=transform,
        dst_crs=src1.crs,
        resampling=Resampling.nearest
        )

        with rasterio.open('/Volumes/Rohan/Mars_GIS_Data/THEMIS/thermal_inertia/THEMIS_TI_Mosaic_Quant_60S300E_100mpp_ESRI104971.tif', 'w', **kwargs) as dst:
            dst.write(reprojected_data)
        
        src2 = rasterio.open('/Volumes/Rohan/Mars_GIS_Data/THEMIS/thermal_inertia/THEMIS_TI_Mosaic_Quant_60S300E_100mpp_ESRI104971.tif')

# For previous Hellas obs

In [1]:
import math

def mars_radius(latitude):
    """
    Calculate the radius of Mars at a given latitude, assuming Mars is an oblate ellipsoid.

    Parameters:
    latitude (float): Latitude in degrees.

    Returns:
    float: Radius of Mars at the specified latitude in kilometers.
    """
    # Equatorial and polar radii of Mars in kilometers
    equatorial_radius = 3396.19  # Semi-major axis (a)
    polar_radius = 3376.2       # Semi-minor axis (b)

    # Convert latitude from degrees to radians
    lat_rad = math.radians(latitude)

    # Formula for the radius of an oblate ellipsoid at a given latitude
    numerator = (equatorial_radius**2 * math.cos(lat_rad))**2 + (polar_radius**2 * math.sin(lat_rad))**2
    denominator = (equatorial_radius * math.cos(lat_rad))**2 + (polar_radius * math.sin(lat_rad))**2
    radius = math.sqrt(numerator / denominator)

    return radius

# Example usage:
latitude = -42.4  # Latitude in degrees
print(f"Radius of Mars at latitude {latitude}°: {mars_radius(latitude):.2f} km")


Radius of Mars at latitude -42.4°: 3387.17 km


In [9]:
import os
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import pandas as pd
from tqdm import tqdm
from shapely.wkt import loads
from osgeo import osr
from pyproj import CRS
import math

os.environ['PROJ_IGNORE_CELESTIAL_BODY'] = 'YES'

# Function to reproject raster to match CRS of polygons
def reproject_raster_to_match_polygons(src, polygons_crs):
    if src.crs != polygons_crs:
        # Reproject raster to match polygons CRS
        transform, width, height = calculate_default_transform(src.crs, polygons_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': polygons_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        reprojected_data = np.empty((src.count, height, width), dtype=src.dtypes[0])
        reproject(
            source=src.read(),
            destination=reprojected_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=polygons_crs,
            resampling=Resampling.nearest
        )

        return reprojected_data, kwargs
    else:
        return src.read(), src.meta

# Radius of Mars in kilometers at Argyre (assuming ellipsoid with 3396.19, 3376.2)
hellas_center_point = (-42.4, 70.5)
R_MARS = mars_radius(hellas_center_point[0])

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on Mars (in kilometers)
    given their latitudes and longitudes.
    
    :param lat1: Latitude of the first point in degrees
    :param lon1: Longitude of the first point in degrees
    :param lat2: Latitude of the second point in degrees
    :param lon2: Longitude of the second point in degrees
    :return: Distance between the two points in kilometers
    """
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance in kilometers
    d = R_MARS * c
    return d

def custom_distance(p1, p2):
    lon1, lat1 = p1.x, p1.y
    lon2, lat2 = p2.x, p2.y
    return round(haversine(lon1, lat1, lon2, lat2), 2)

# Function to calculate representative point of each polygon
def calculate_representative_point(polygons, key_):
    polygons[f'representative_point_{key_}'] = polygons['geometry'].representative_point()
    return polygons

aug_notes = ['+HCP', '+LCP', '+Olivine', '+Plagioclase']

rename_dict = {'Plagioclase+Olivine': 'Olivine+Plagioclase', 
               'HCP+LCP': 'LCP+HCP',
               'LCP+Olivine': 'Olivine+LCP',
               'HCP+Olivine': 'Olivine+HCP', 'HCP+Plagioclase': 'Plagioclase+HCP',}

color_map_dict = {'Olivine': 'red', 'Plagioclase': 'yellow', 
                  'LCP': 'cyan', 'HCP': 'magenta', 
                  'Olivine+LCP': 'lavender', 'Olivine+HCP': 'purple', 'Olivine+Plagioclase': 'orangered',
                  'LCP+HCP': 'blue'}

# Function to calculate mean of raster over each polygon
def calculate_metadata(polygons, src, raster_meta, geounits, single_point, source_file, crs):

    # List to store results
    results = []

    # Loop over each polygon
    for idx, row in tqdm(polygons.iterrows(), total=len(polygons), desc="Calculating mean raster"):
        # Mask raster by each polygon
        try:
            out_image, out_transform = mask(src, [row.geometry], crop=True)
        except ValueError:
            continue
        
        out_image = np.where(out_image < -8210, np.nan, out_image)
        # Calculate mean
        mean_value = np.nanmean(out_image)

        # Calculate distance from polygon's representative point to single point
        p = row['representative_point_distance']
        note = row['Notes']
        if note in aug_notes:
            cat_plus = note
        else:
            cat_plus = ''
        
        distance = custom_distance(single_point, p)

        cat = row['Interpreta'] + cat_plus

        if cat in rename_dict.keys():
            cat = rename_dict[cat]

        color = color_map_dict[cat]

        geounits_gdf = gpd.read_file(geounits)
        p = row['representative_point_units']
        intersecting_features = geounits_gdf[geounits_gdf.intersects(p)]
        try:
            unit_name = intersecting_features['Unit'].values[0]
        except:
            unit_name = 'None'
        # if idx % 10 == 0:
        #     print(f"Unit name: {unit_name}")
        # Append results
        results.append({
            'Source File': source_file,
            'Polygon_ID': idx,
            'Color': color,
            'Category': cat.lower(),
            'Representative Point': row['representative_point_distance'],
            'Elevation_km': mean_value/1000,
            'Distance_to_center_km': distance,
            'GeoUnits': unit_name
        })

    # Create a DataFrame from results
    df = pd.DataFrame(results)

    return df

# Function to save results to Excel
def save_to_excel(df, excel_file):
    df.to_excel(excel_file, index=False, engine='openpyxl')

# Function to process all gpkg files in a folder
def process_gpkg_folder(gpkg_folder, raster_file, geounits, point_shapefile, output_excel, out_gpkg):
    all_results = []
    # Define Mars CRS
    mars_crs_simple = {
        'proj': 'longlat',
        'ellps': 'sphere',
        'a': 3396190,  # semi-major axis
        'b': 3396190,  # semi-minor axis
        'units': 'm',
        'no_defs': True
    }

    # Read the single point from shapefile into a GeoDataFrame
    single_point_gdf = gpd.read_file(point_shapefile)

    # Reproject the GeoDataFrame to the desired CRS
    single_point_gdf = single_point_gdf.to_crs(mars_crs_simple)
    
    # Extract the reprojected geometry
    single_point = single_point_gdf.iloc[0]['geometry']

    # Print single point CRS for debugging
    print(f"Single point CRS: {single_point_gdf.crs}")

    # Open raster file
    with rasterio.open(raster_file) as src:
        raster_data, raster_meta = reproject_raster_to_match_polygons(src, polygons_crs=src.crs)

        # List all .gpkg files in the folder
        gpkg_files = [os.path.join(gpkg_folder, file) for file in os.listdir(gpkg_folder) if file.endswith('.gpkg')]

        # Process each .gpkg file
        for gpkg_file in gpkg_files:
            print(f"Processing {gpkg_file}...")
            # Read the polygon geometries from the .gpkg file
            polygons = gpd.read_file(gpkg_file)

            # Calculate representative point for each polygon
            p_crs = polygons.crs
            polygons = polygons.to_crs(mars_crs_simple)
            polygons = calculate_representative_point(polygons, key_ = 'distance')
            polygons = polygons.to_crs(p_crs)
            polygons = calculate_representative_point(polygons, key_ = 'units')

            source_file = gpkg_file.split('.')[0].split('/')[-1]
            polygons = polygons.to_crs(src.crs)  # Ensure polygons are in the raster CRS

            results_df = calculate_metadata(polygons, src, raster_meta, geounits, single_point, source_file, mars_crs_simple)
            all_results.append(results_df)

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(all_results, ignore_index=True)

    # Save results to Excel
    save_to_excel(final_df, output_excel)

    # Save the updated GeoDataFrame back to the original .gpkg file
    # final_df['geometry'] = final_df['Representative Point']
    gdf = gpd.GeoDataFrame(final_df, geometry = 'Representative Point')
    gdf.set_crs(mars_crs_simple)
    gdf.to_file(out_gpkg, driver='GPKG')

# usage
DATE_STRING = '12042024'
if __name__ == "__main__":
    # Folder containing .gpkg files
    gpkg_folder = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Hellas/2022_paper/collated/'
    # gpkg_folder = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/targeted/categorized/'

    # Path to raster file
    raster_file = '/Volumes/Rohan/Mars_GIS_Data/MOLA_HRSC/Mars_HRSC_MOLA_BlendDEM_Global_200mp_v2.tif'

    # Path to single point shapefile
    point_shapefile = '/Volumes/Valaquenta/Desktop/Hellas_center_point.shp'

    # Path to geo units file
    geounits = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Hellas/2022_paper/north_hellas_geounits.gpkg'

    # Output Excel file
    # output_excel = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/mineral_unit_metadata_categorized_11042024.xlsx'
    output_excel = f'/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Hellas/2022_paper/hellas_mineral_unit_metadata_categorized_{DATE_STRING}.xlsx'

    # output geopackage file
    # out_gpkg = '/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Argyre/all_units_points_argyrePRJ_11042024.gpkg'
    out_gpkg = f'/Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Hellas/2022_paper/hellas_all_units_points_argyrePRJ_{DATE_STRING}.gpkg'
    
    # Process all .gpkg files in the folder
    process_gpkg_folder(gpkg_folder, raster_file, geounits, point_shapefile, output_excel, out_gpkg)



Single point CRS: +proj=longlat +ellps=sphere +a=3396190 +b=3396190 +units=m +no_defs +type=crs
Processing /Users/phillipsm/Documents/Research/MassifMapping/Argyre_QGIS/shape_files/Hellas/2022_paper/collated/north_hellas_mafics_.gpkg...


Calculating mean raster:  97%|█████████▋| 981/1016 [03:28<00:07,  4.90it/s]/var/folders/fz/2g6_hdjd4yndg54gmwq1402c0000gn/T/ipykernel_21286/2728466048.py:111: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(out_image)
Calculating mean raster: 100%|██████████| 1016/1016 [03:34<00:00,  4.73it/s]
